In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

#import pandas as pd

import sys

import os

sys.path.insert(0, "/Users/jonathan/wine_analysis_hplc_uv/scripts/agilette/")

import agilette

ag = agilette.Agilette('/Users/jonathan/0_jono_data')

In [ ]:
ag.data_table().head()

## Loading the Acetone Void Vol Data

In [ ]:
acetone_data = ag.data_file_dir.sequence_dict['2023-03-01_15-22-02_ACETONE_VOID-VOL-MEASUREMENT.sequence'].data_dict

## Extracting Abs Maxima Wavelength Signal

In [ ]:
acetone_data['acetone0002'].extract_ch_data()['DAD1B.ch']

Acetone has a abs max around 279nm, which is designated as Signal B:

In [ ]:
signal = 'DAD1B.ch'

## Renaming The Runs

In [ ]:
string_to_split = 'acetone0001'

In [ ]:
left_str = string_to_split[len(string_to_split)-1:]

right_str = string_to_split[:-4] 

left_str + "_" + right_str

In [ ]:
acetone_data_sigmax = {}

for key, value in acetone_data.items():
    reform_key = key[:-4] + "_" + key[len(key)-4:]
    acetone_data_sigmax[reform_key] = value.extract_ch_data()['DAD1B.ch']

In [ ]:
acetone_data_sigmax

## Plotting the Runs

In [ ]:
sys.path.insert(0, "/Users/jonathan/wine_analysis_hplc_uv/scripts")

In [ ]:
import pandas as pd

from scipy.signal import find_peaks

In [ ]:
import plotly.graph_objs as go

## set up parameters for peak detection and plotting.

run_name = 'acetone_0001'

data = acetone_data_sigmax[f'{run_name}']

a_1 = pd.DataFrame(data[:,1], columns = ['mAU'], index = data[:,0])

### Prototyping the Plot

In [ ]:
# set up the figure

nm = '279nm'

fig = go.Figure()

plot_title = f"{run_name}"

fig.update_layout(title = f"{plot_title}, {nm}")

# set chromatogram x and y values.

cx = data[:,0]

cy = data[:,1]

import numpy as np

# # draw the chromatogram

chrom_trace = go.Scatter(x = cx, y = cy, mode = 'lines', name = 'chromatogram')

fig.add_trace(chrom_trace)

# find peaks

peak_idx, peak_heights = find_peaks(a_1['mAU'], height = 50, distance = 50)

px = a_1.index.values[peak_idx].tolist()[0]

py = a_1.values[peak_idx][0].tolist()[0]

# draw peaks

peak_trace = go.Scatter(x = [px], y = [py], mode = 'markers', name = f'{np.round(px, 2)} mins, {int(py)} mAU')

fig.add_trace(peak_trace)

fig.show()

So, what do we want to do now? Get the peak description for each reading, then plot their values against concentration. First make a function doing the same.

### Functionalising it

In [ ]:
import numpy as np

def voidvolplotter(data_df, run_name):
    
    # set up the figure

    fig = go.Figure()

    fig.update_layout(title = f"{run_name}, {nm}")

    # set chromatogram x and y values.

    cx = data_df['mins'].tolist()

    cy = data_df['mAU'].tolist()

    # # draw the chromatogram

    chrom_trace = go.Scatter(x = cx, y = cy, mode = 'lines', name = 'chromatogram')

    fig.add_trace(chrom_trace)

    # find peaks

    peak_idx, peak_heights = find_peaks(data_df['mAU'], height = 50, distance = 50)

    px = data_df.index.values[peak_idx].tolist()
    
    print(type(px))

    py = data_df.values[peak_idx].tolist()

    # draw peaks
    peak_trace = go.Scatter(x = px, y = py, mode = 'markers', name = f'{np.round(px[0], 2)} mins, {py[0][0]} mAU')

    fig.add_trace(peak_trace)
    
    return fig

In [ ]:
## Forming a Better Data Structure

In [ ]:
inj_vol = {
    "1_acetone" : 10,
    "2_acetone" : 8,
    "3_acetone" : 6,
    "4_acetone" : 4,
    "5_acetone" : 2,
    "6_acetone" : 1
}

{ "name" : { inj_vol : data }}

In [ ]:
acetone_dict = {}

for run_name, inj_vol_ in inj_vol.items():
    for run_name_1, data in acetone_data_sigmax.items():
        
        data = pd.DataFrame(data, columns = ['mins', 'mAU'])
        
        acetone_dict[run_name] = { "inj_vol" : inj_vol_, "data" : data}

acetone_dict['1_acetone']

So the pattern is:

acetone_dict['run_name']['inj_vol'], or acetone_dict['run_name']['data']

In [ ]:
for run_name, contents in acetone_dict.items():

    print(run_name)
    print(contents['inj_vol'])

    print(find_peaks(contents['data']['mAU'], height = 50))

This is a mess, let's redefine. Maybe OOP. But need to know in what format the data needs to be.

For the plot to work I need:

chromatogram as a df, injection vol, run name.

In [ ]:
class Void_Vol_Measurement:
    def __init__(self, run_name, injection_vol, data):
        self.run_name = run_name
        self.inj_vol = injection_vol
        self.data = data

In [ ]:
run_vol_zip = zip(acetone_data.keys(), inj_vol.values())

void_vol_measure_obj_list = []

for run, injvol in run_vol_zip:
    void_vol_measure_obj_list.append(Void_Vol_Measurement(run, inj_vol, acetone_data[run]))